In [1]:
##################################################################################################################
##################################################################################################################
###############################################ngrams representation###############################################
##################################################################################################################
##################################################################################################################

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import gensim
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk

from pprint import pprint

import spacy

import pickle
import re 
import pyLDAvis
import pyLDAvis.gensim_models

In [3]:
import os
cwd = os.getcwd()
cwd

'C:\\Users\\Wahbeh\\Desktop\\Vaccine Topics - Python'

In [4]:
df = pd.read_csv ('sample.csv')
df

,Full Text
0,@hamcore_ First the boss metal zone schematics...
1,@sippygup its in the vaccine and 5g makes it m...
2,Him and all the others that do Not like and ha...
3,@TimRunsHisMouth Him and all the others that d...
4,@NYGovCuomo 99.97% survival rate no vaccine fo...
...,...
9953,Waiting for the train and I hear two mfs say �...
9954,I played my radical left avant garde music and...
9955,You are such a fag @MayorKeller And what the F...
9956,.@seanspicer hahahaha Trump's accomplishments?...


In [5]:
docs = np.array(df['Full Text'])

In [6]:
for i in range(0,9958):
    df.iloc[i, 0] = " ".join(filter(lambda x:x[0]!='#', df.iloc[i, 0].split()))
    df.iloc[i, 0] = " ".join(filter(lambda x:x[0]!='@', df.iloc[i, 0].split()))
df  

,Full Text
0,First the boss metal zone schematics as 5g in ...
1,its in the vaccine and 5g makes it more powerf...
2,Him and all the others that do Not like and ha...
3,Him and all the others that do Not like and ha...
4,99.97% survival rate no vaccine for me
...,...
9953,Waiting for the train and I hear two mfs say �...
9954,I played my radical left avant garde music and...
9955,You are such a fag And what the Fuck is that q...
9956,.@seanspicer hahahaha Trump's accomplishments?...


In [7]:
docs = np.array(df['Full Text'])
docs

array(['First the boss metal zone schematics as 5g in a vaccine now hitler got stuff right??? Cmon man it�s only 10:42am',
       'its in the vaccine and 5g makes it more powerful watch out',
       "Him and all the others that do Not like and hate DonaldTrump is just because They are part of the satanic Deep State illuminati and serves different??God, the same God that gave him the power to poisoning our food and water. That's Monsanto plus he is behind the Vaccines?? withVP",
       ...,
       'You are such a fag And what the Fuck is that que que logo all about? What a stupid waste of time and money you are. No mask No distancing No testing No vaccine',
       ".@seanspicer hahahaha Trump's accomplishments?? The vaccine? That international companies have been working on since before Trump acknowledged there was a pandemic? The economy? It rallied when Biden won and now when the transition started. Trump killed farmers.",
       "100%. It was about the election, not the virus. Now it

In [8]:
print(docs.shape)

(9958,)


# Pre-processing

In [9]:
from nltk.tokenize import TweetTokenizer

nltk.download('stopwords')
# get a set of stopwords
stopwords = set(nltk.corpus.stopwords.words("english"))

from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Wahbeh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Wahbeh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
def docs_preprocessor(docs):
    # Tokenize corpus
    tokenizer = nltk.tokenize.RegexpTokenizer(r"[a-z]+")

    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.     
        

    # Only keep tokens that contains alphanumeric characters, without symbols.
    docs = [[token for token in doc if token.isalnum()] for doc in docs]
    
    # Remove stopwords.
    docs = [[token for token in doc if token not in stopwords] for doc in docs]

    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 1] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs

In [11]:
docs = docs_preprocessor(docs)

In [12]:
docs

[['first',
  'bos',
  'metal',
  'zone',
  'schematic',
  'vaccine',
  'hitler',
  'got',
  'stuff',
  'right',
  'cmon',
  'man'],
 ['vaccine', 'make', 'powerful', 'watch'],
 ['others',
  'like',
  'hate',
  'donaldtrump',
  'part',
  'satanic',
  'deep',
  'state',
  'illuminati',
  'serf',
  'different',
  'god',
  'god',
  'gave',
  'power',
  'poisoning',
  'food',
  'water',
  'monsanto',
  'plus',
  'behind',
  'vaccine',
  'withvp'],
 ['others',
  'like',
  'hate',
  'donaldtrump',
  'part',
  'satanic',
  'deep',
  'state',
  'illuminati',
  'serf',
  'different',
  'god',
  'god',
  'gave',
  'power',
  'poisoning',
  'food',
  'water',
  'monsanto',
  'plus',
  'behind',
  'vaccine',
  'withvp'],
 ['survival', 'rate', 'vaccine'],
 ['coincidence',
  'extremely',
  'rare',
  'say',
  'pro',
  'vaxxers',
  'like',
  'autism',
  'million',
  'family',
  'dealing',
  'least',
  'one',
  'child',
  'autism',
  'sometimes',
  'mmr',
  'vaccine',
  'relation',
  'injury',
  'rare'],

In [13]:
processed_docs = []
for i in range(len(docs)):
    processed_docs.append(' '.join(docs[i]))

In [14]:
processed_docs

['first bos metal zone schematic vaccine hitler got stuff right cmon man',
 'vaccine make powerful watch',
 'others like hate donaldtrump part satanic deep state illuminati serf different god god gave power poisoning food water monsanto plus behind vaccine withvp',
 'others like hate donaldtrump part satanic deep state illuminati serf different god god gave power poisoning food water monsanto plus behind vaccine withvp',
 'survival rate vaccine',
 'coincidence extremely rare say pro vaxxers like autism million family dealing least one child autism sometimes mmr vaccine relation injury rare',
 'part vaccination chip implant wake people',
 'wait get better coverage covid vaccine',
 'believe vaccine cause autism unfollow',
 'method vaccine distribution ethic mandated vaccine topic debate exploration one teacher present impartially whether vaccine cause autism clear truth evidence vaccine cause autism',
 'insurrection antifa widespread voter fraud vaccine cause autism covid overblown hoax 

In [15]:
len(processed_docs)

9958

In [16]:
from nltk import ngrams
ngram_list = []
for i in range(len(processed_docs)):
    doc = processed_docs[i]
    bigram = ngrams(doc.split(' '), n=2)
    trigram = ngrams(doc.split(' '), n=3)
    ngram_docs = []
    for x in bigram:
        ngram_docs.append(' '.join(x))
    for y in trigram:
        ngram_docs.append(' '.join(y))
    ngram_list.append(ngram_docs)

In [17]:
ngram_list[0]

['first bos',
 'bos metal',
 'metal zone',
 'zone schematic',
 'schematic vaccine',
 'vaccine hitler',
 'hitler got',
 'got stuff',
 'stuff right',
 'right cmon',
 'cmon man',
 'first bos metal',
 'bos metal zone',
 'metal zone schematic',
 'zone schematic vaccine',
 'schematic vaccine hitler',
 'vaccine hitler got',
 'hitler got stuff',
 'got stuff right',
 'stuff right cmon',
 'right cmon man']

In [18]:
docs = ngram_list

In [19]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
docs_dict = Dictionary(docs)
print('Number of unique words in initital documents:', len(docs_dict))

# # Filter out words that occur less than 10 documents, or more than 40% of the documents.
# docs_dict.filter_extremes(no_below=4, no_above=0.40)
# print('Number of unique words after removing rare and common words:', len(docs_dict))

Number of unique words in initital documents: 185404


In [20]:
#bag of words representation
corpus = [docs_dict.doc2bow(doc) for doc in docs]
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1)]

In [21]:
corpus[55]

[(1385, 1), (1386, 1), (1387, 1), (1388, 1), (1389, 1), (1390, 1), (1391, 1)]

In [22]:
print('Number of unique tokens: %d' % len(docs_dict))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 185404
Number of documents: 9958


# LDA topic model

In [23]:
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel

In [24]:
# Set training parameters.
num_topics = 10
chunksize = 9958 # size of the doc looked at every pass
passes = 4 # number of passes through documents
iterations = 1
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = docs_dict[0]  # This is only to "load" the dictionary.
id2word = docs_dict.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

Wall time: 12 s


In [25]:
#Print all the topics
for i in range(0, model.num_topics):
    print (model.print_topic(i))
    print("-------------------------------------------------------------")

0.003*"deep state" + 0.002*"vaccine cause" + 0.002*"vaccine cause autism" + 0.001*"cause autism" + 0.001*"covid vaccine" + 0.001*"bill gate" + 0.001*"take vaccine" + 0.001*"side effect" + 0.001*"vaccine safe" + 0.001*"get vaccine"
-------------------------------------------------------------
0.004*"deep state" + 0.002*"vaccine cause" + 0.002*"cause autism" + 0.002*"covid vaccine" + 0.001*"vaccine cause autism" + 0.001*"take vaccine" + 0.001*"get vaccine" + 0.001*"belong free society" + 0.001*"unconstitutional medical mandate" + 0.001*"free vaccine"
-------------------------------------------------------------
0.005*"deep state" + 0.002*"vaccine cause" + 0.002*"vaccine cause autism" + 0.002*"cause autism" + 0.001*"take vaccine" + 0.001*"covid vaccine" + 0.001*"get vaccine" + 0.001*"bill gate" + 0.001*"side effect" + 0.001*"get vaccinated"
-------------------------------------------------------------
0.003*"deep state" + 0.002*"vaccine cause autism" + 0.002*"vaccine cause" + 0.002*"cause

In [26]:
# Compute model Perplexity - a measure of how good the model is. lower the better`
print('\nPerplexity: ', model.log_perplexity(corpus,total_docs=9958))  

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=model, texts=docs, dictionary=docs_dict , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -13.57634320108277

Coherence Score:  0.3992865164091464


# Optimizing LDA model

In [27]:
def compute_coherence_values(id2word, dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    # Fixed LDA model parameters
    chunksize = 9958 # size of the doc looked at every pass
    passes = 4 # number of passes through documents
    iterations = 1
    # eval_every = 1  # Don't evaluate model perplexity, takes too much time.

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        print("LDA model with ", num_topics, "topics")
        model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes)
        model_list.append(model)
        
        print("Calculating coherence score")
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(id2word=id2word, dictionary=docs_dict, \
                                                        corpus=corpus, texts=docs, start=2, limit=51, step=1)

LDA model with  2 topics
Calculating coherence score
LDA model with  3 topics
Calculating coherence score
LDA model with  4 topics
Calculating coherence score
LDA model with  5 topics
Calculating coherence score
LDA model with  6 topics
Calculating coherence score
LDA model with  7 topics
Calculating coherence score
LDA model with  8 topics
Calculating coherence score
LDA model with  9 topics
Calculating coherence score
LDA model with  10 topics
Calculating coherence score
LDA model with  11 topics
Calculating coherence score
LDA model with  12 topics
Calculating coherence score
LDA model with  13 topics
Calculating coherence score
LDA model with  14 topics
Calculating coherence score
LDA model with  15 topics
Calculating coherence score
LDA model with  16 topics
Calculating coherence score
LDA model with  17 topics
Calculating coherence score
LDA model with  18 topics
Calculating coherence score
LDA model with  19 topics
Calculating coherence score
LDA model with  20 topics
Calculatin

In [ ]:
# Show graph
limit=51; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()# Print the coherence scores

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# type(model_list)
# len(model_list)
# print("Coherence score = ", coherence_values[43])
# model_list[43]

In [ ]:
# Optimal Model
num_topics = 47
chunksize = 9958 # size of the doc looked at every pass
passes = 6 # number of passes through documents
iterations = 1
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = docs_dict[0]  # This is only to "load" the dictionary.
id2word = docs_dict.id2token

%time optimal_model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

#Print all the topics
for i in range(0, optimal_model.num_topics):
    print ('Topic', i , ' ', optimal_model.print_topic(i))
    print("-------------------------------------------------------------")

# Visualize 'optimal' LDA model

In [ ]:
import pyLDAvis
pyLDAvis.gensim_models
import pyLDAvis.gensim_models
import warnings
pyLDAvis.enable_notebook()
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
#pcoa:Principal Coordinate Analysis (aka Classical Multidimensional Scaling)
pyLDAvis.gensim_models.prepare(optimal_model, corpus, docs_dict)

In [ ]:
# mmds:Metric Multidimensional Scaling
pyLDAvis.gensim_models.prepare(optimal_model, corpus, docs_dict, mds='mmds')

In [ ]:
#tsne:t-distributed Stochastic Neighbor Embedding
pyLDAvis.gensim_models.prepare(optimal_model, corpus, docs_dict, mds='tsne')